In [1]:
import plotly.express as px

# Library for data manipulation and analysis
import pandas as pd

# Importing dash componants
import dash
from jupyter_dash import JupyterDash

# For using dash components
from dash import dcc

# use of html tags like div
from dash import html

# For call back
from dash.dependencies import Output, Input

# For displaying the structural elements
import dash_bootstrap_components as dbc

In [3]:
# Load data
df = pd.read_excel(r'C:\Users\archa\Documents\Archana\IIS_GoogleDrive\IIS\Thesis\Julius\Application\Data\Data.xlsx',
                   header=1,skiprows=1, names=['Roles','Actors','Resources','Activities','value contribution in ecosystem','Value contribution for the actors','Dependency','RoleInfo'])

# Saving 4 elements into single column. This makes it easy to display the data
df = pd.melt(df, id_vars=['Roles','Actors','Dependency','RoleInfo'], 
             value_vars=['Resources','Activities','value contribution in ecosystem','Value contribution for the actors'],
             var_name='results',value_name='StructuralEements')


In [4]:
# Build App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

app.layout = dbc.Container([
#     Creating treemap 
    dcc.Graph(id='treemap',
             figure= px.treemap(df,
                                 path=[px.Constant('Ecosystem'),'Roles','Actors'],
                                values='Dependency',
                                 height=600, width=1450).update_layout(margin=dict(t=25, r=0, l=5, b=20))
              ),
#     Div to display the elements. This will be empty till a brach is cliked in treemap
    dbc.Row([
    dbc.Col([html.Div(id='Elementscontainer')], width=12)
    ])

], fluid=True)

# Define callback to display the structural elememts
@app.callback(   
    Output("Elementscontainer", "children"),
    Input("treemap","clickData")
)

def update_modal(data):
    #if black frame of treemap, don't update    
    if data is None:
        return dash.no_update 
    
    #if no currentpath or ecosystem is chosen, don't update   
    elif data['points'][0].get('currentPath') is None \
            or data['points'][0]['percentRoot'] == 1: 
        table_header = " "
        table_body = " "
        return dbc.Table(table_header + table_body, bordered=False)
    
    #if Role is chosen (ecosystem is parent), don't update
    elif data['points'][0]['label'] in df.Roles.unique():         
        label_slct = data['points'][0]['label']
        parent_slct = data['points'][0]['parent'] 
        #Filtering data for the seleced role
        dff = df[(df.Roles == label_slct)]         
        # create the table
        table_header = [html.Thead(html.Tr([html.Th(label_slct)]))]
        table_body = [html.Tbody(dff["RoleInfo"].unique())]
        return dbc.Table(table_header + table_body, bordered=True)
    
    # if Actor is chosen, build table 
    else:        
        label_slct = data['points'][0]['label']
        parent_slct = data['points'][0]['parent']   
        #Filtering data for selected actor
        dff = df[(df.Roles == parent_slct) & (df["Actors"] == label_slct)]        
        # create the table
        table_header = [html.Thead(html.Tr([html.Th(label_slct)]))]
        rows=[]
        for x in dff["StructuralEements"]:
            if pd.isnull(x):
                continue 
            rows.append(html.Tr([html.Td(x)]))
        table_body = [html.Tbody(rows)]
        return dbc.Table(table_header + table_body, bordered=True)
    
app.run_server(mode='external', port=8025)
# app.run_server(mode='inline')

Dash app running on http://127.0.0.1:8025/
